<a href="https://colab.research.google.com/github/vinayshanbhag/AustinRadarTraffic/blob/master/Austin_Traffic_Radar_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Austin Radar Traffic Sensor data - Visualizing time series data on a map

Data Source: City of Austin https://data.austintexas.gov/Transportation-and-Mobility/Radar-Traffic-Counts/i626-g7ub

In [0]:
import pandas as pd
import folium
from folium import plugins
import requests, json
from datetime import datetime
from scipy.interpolate import interp1d

from google.colab import drive

#### Load traffic volume data

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
f = '/content/drive/My Drive/Radar_Traffic_Counts.csv'

In [0]:
df = pd.read_csv(f)

#### Load sensor location data

In [0]:
filter_clause = '$where='
for idx, val in enumerate(df['KITS ID'].unique()):
  if idx == 0: filter_clause = filter_clause + f'kits_id={val}'
  else:filter_clause = filter_clause + f' or kits_id={val}'

In [0]:
sensors = requests.get(f"https://data.austintexas.gov/resource/6yd9-yz29.json?{filter_clause}").json()

In [0]:
sensors = [s for s in sensors if s.get('kits_id','')]

In [0]:
sensors = {int(s['kits_id']):s for s in sensors if int(s['kits_id']) in df['KITS ID'].unique()}

#### Prepare data

<p>
Drop sensor data for any sensors we can't locate.

Add location names and geo codes.

Aggregate traffic volumes by hour of day for each location.
</p>

In [0]:
df = df[df['KITS ID'].isin(sensors.keys())].copy()

In [0]:
df['location_name'] = df['KITS ID'].apply(lambda x: sensors[x]['location_name'])
df['location_latitude'] = df['KITS ID'].apply(lambda x: float(sensors[x]['location_latitude']))
df['location_longitude'] = df['KITS ID'].apply(lambda x: float(sensors[x]['location_longitude']))

In [17]:
hourly_vol_by_loc = df.groupby(['location_name','location_latitude','location_longitude','Hour']).agg({'Volume':'sum'}).reset_index()
hourly_vol_by_loc.sample()

,location_name,location_latitude,location_longitude,Hour,Volume
15,BURNET RD / PALM WAY (IBM DRIVEWAY),30.402286,-97.717606,15,950700


#### Construct Time series Geo JSON and map

In [18]:
# Center map between lat/lon extrema
min_lat, max_lat, min_lon, max_lon = hourly_vol_by_loc.location_latitude.min(),hourly_vol_by_loc.location_latitude.max(),hourly_vol_by_loc.location_longitude.min(),hourly_vol_by_loc.location_longitude.max()
center_lat, center_lon = (min_lat + (max_lat-min_lat)/2,min_lon + (max_lon-min_lon)/2)

# Date slider - We are visualizing volume at each location by hour of day, using several years of data. Date can be ignored.
dt = datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)

# set point size on map based on actual volume
m = interp1d([hourly_vol_by_loc.Volume.min(),hourly_vol_by_loc.Volume.max()],[5,20])

# construct geo json
data = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "geometry": {
        "type": "MultiPoint",
        "coordinates": [[row[2],row[1]]],
      },
      "properties": {
        "icon":"circle",
        'iconstyle':{
          'fillColor': 'red' if row[4] > hourly_vol_by_loc[(hourly_vol_by_loc['location_name']==row[0])].Volume.mean() else 'green',
          'fillOpacity': 0.6,
          'stroke': False,
          'radius': int(m(row[4]))#/hourly_vol_by_loc.Volume.max()*20),
        },
        "times": [dt.replace(hour=row[3]).strftime("%Y-%m-%dT%T")]
      }
    }
    for idx, row in enumerate(hourly_vol_by_loc.values)
  ]
}

m = folium.Map([center_lat, center_lon], zoom_start=12)
m.add_child(plugins.TimestampedGeoJson(data
                                       ,period='PT1H'
                                       , transition_time=1000
                                       , duration='PT1M'
                                       ,add_last_point=False)
)
m